<a href="https://colab.research.google.com/github/FabiolaAJ/chatbot-assistente-de-vagas/blob/main/Chatbot_Assistente_de_Vagas_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
!pip install -q google-adk

In [178]:
import os
from google.colab import userdata
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
from IPython.display import clear_output

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [179]:
##########################################
# --- Agente 1: Buscador de Vagas --- #
##########################################
def agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente simpático especializado na busca de vagas de emprego.
        Use a ferramenta de busca do google (google_search) para encontrar até 5 vagas relevantes e atuais para a profissão e senioridade do usuário.

          1. **Interpretação:** Analise profissão e senioridade. Se a senioridade faltar, pergunte.
          2. **Busca (google_search):** Combine profissão, área, senioridade e informe se a vaga é hibrida, presencial ou remoto. Foque em resultados que provavelmente contenham links diretos para a página da vaga.
          3. **Filtragem:** Priorize vagas recentes verificando a data de hoje (data_de_hoje). Descarte vagas antigas ou encerradas.
          4. **Resposta (máximo 5 vagas):** Para cada vaga, retorne título, empresa, localização (se disponível), breve descrição com requisitos para vaga, não faça questionamentos adicionais para o usuário.
          5. **Link:** [**Link direto para a página da vaga**. Este é um campo obrigatório. Tente ao máximo encontrar o link nos resultados da busca.]
          6. **Sem Resultados:** Se não houver vagas relevantes e atuais, informe o usuário.

          Priorize informações precisas e atuais.

        """,
        description="Agente que busca informações de vagas de empregos no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Profissão: {profissao} \n Area:{area} \n Senioridade:{senioridade} \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [180]:
################################################
# --- Agente 2: Planejador de carreira e recrutamento --- #
################################################
def agente_planejador_de_carreira(profissao, area, senioridade, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em carreira e recrutamento muito gentil e simpático, com vasta experiência em ajudar candidatos a se prepararem para processos seletivos.
        Gere dicas para se destacar, considerando os seguintes tópicos:
             1. Quais habilidades e experiências o candidato deve enfatizar?
             2. Forneça até 2 dicas concisas sobre como o usuário pode se organizar para se candidatar as vagas (ex: adaptar currículo, pesquisar a empresa).
             3. Ao final cite dicas gerais de como se destacar no mercado de trabalho de acordo com a profissão do usuário
        """,
        description="Agente planejador especialista em carreira",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Profissão: {profissao} \n Area:{area}  \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados}"

    # Executa o agente
    lancamento_planejamento = call_agent(planejador, entrada_do_agente_planejador)
    return lancamento_planejamento

In [186]:
def agente_responder_duvidas(profissao, area, senioridade, duvida, lancamentos_buscados, lancamento_planejamento):

    buscador = Agent(
        name="agente_buscador_de_vagas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de carreira e você sempre responde de forma gentil e de facil entendimento.
        Use a ferramenta de busca do google (google_search) para tirar a dúvida do usuário sobre sua carreira, responda de forma resumida.
        """,
        description="Agente para tirar dúvidas de carreira",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Dúvida: {duvida}\n Profissao:{profissao} \n Area:{area} \n Senioridade:{senioridade} \n Buscados: {lancamentos_buscados} \n Planejador: {lancamento_planejamento}  \n Data de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [182]:
def obter_input_obrigatorio(mensagem_inicial, mensagem_erro):
    # Exibe a mensagem inicial para o usuário, solicitando a entrada.
    valor = input(f"\n{mensagem_inicial} ")

    # Inicia um loop 'while' que continuará executando enquanto a condição for verdadeira.
    # A condição 'not valor.strip()' verifica se a variável 'valor' está vazia ou contém apenas espaços em branco.
    # 'valor.strip()' remove os espaços em branco do início e do final da string.
    # Se após a remoção dos espaços, a string estiver vazia, a condição será 'True' e o loop continuará.
    while not valor.strip():
        # Exibe a mensagem de erro informando ao usuário que a entrada é obrigatória.
        print(f"{mensagem_erro}")

        # Solicita novamente a entrada do usuário, exibindo a mensagem inicial novamente.
        valor = input(f"\n{mensagem_inicial}")

    # Quando o loop 'while' terminar (ou seja, o usuário digitou algo que não é apenas espaço),
    # a função retorna o valor digitado, com quaisquer espaços em branco iniciais e finais removidos
    # usando o método 'strip()'.
    return valor.strip()

In [207]:
def tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador):
    questionario = "S"

    while questionario.upper() == "S" :
      print("\n Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') \n")
      duvida = input("")
      if(duvida.upper() == 'N'):
        break;
      else:
        print("\n🔄 Processando resposta...")
        resposta_duvida = agente_responder_duvidas(profissao, area, senioridade, duvida, resposta_buscador, resposta_planejador)
        display(to_markdown(resposta_duvida))

        print("\n Se deseja tirar outra dúvida digite 'S'(caso não tenha, digite qualquer tecla): \n")
        questionario = input("")

        if(questionario.upper() != "S"):
          break;


In [184]:
def exibir_resultados(profissao, area, senioridade):
    print(f"")

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    resposta_buscador = agente_buscador_vagas_de_emprego(profissao, area, senioridade, data_de_hoje)
    print("\n 📝 Essas foram as vagas encontradas! \n")
    display(to_markdown(resposta_buscador))
    print("--------------------------------------------------------------")

    print("\n🔄 Processando resultados de planejamento...")
    resposta_planejador = agente_planejador_de_carreira(profissao, area, senioridade, resposta_buscador)
    print("\n 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 \n")
    display(to_markdown(resposta_planejador))
    print("--------------------------------------------------------------")

    tirar_duvidas(profissao, area, senioridade, resposta_buscador, resposta_planejador)

    return

In [211]:
def main():
  exibir_questionario = "1";

  while exibir_questionario == "1" :
    clear_output(wait=True)

    print("\n Olá, tudo bem por aí? 😊 Que bom te ter aqui!")
    print("\n Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉")
    print("\n--------------------------------------------------------------")

    profissao = obter_input_obrigatorio("❓Me conta qual a profissão que te interessa encontrar vagas hoje? ", "/n Hmm, para que eu te ajude a achar as melhores vagas, preciso que me conte qual profissão você busca, tá bem? 😉 ")
    area = obter_input_obrigatorio("❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend) " , "É importante me informar qual área específica dentro dessa profissão deseja atuar!😊 ")
    senioridade = obter_input_obrigatorio("❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...) ", "Porfavor, digite seu nível de experiência para refinar a busca! 😊 ")

    print("\n🔄 Processando resultados...")
    exibir_resultados(profissao, area, senioridade)

    print("\n Se deseja procurar outra vaga, digite 1 (para encerrar digite qualquer tecla): \n")

    exibir_questionario = input("")

    if(exibir_questionario != '1'):
      print("\n Até a próxima! 😊")
      break;


main()




 Olá, tudo bem por aí? 😊 Que bom te ter aqui!

 Sou seu assistente virtual de carreira, pronto para te dar uma mãozinha especial na busca daquela vaga de emprego e te preparar para o sucesso nas entrevistas! 🎉

--------------------------------------------------------------

❓Me conta qual a profissão que te interessa encontrar vagas hoje?  programador

❓Legal! qual a área específica dentro dessa profissão você deseja atuar? (Por exemplo: UX, Gestão, Frontend, Backend)  front end

❓E qual o seu nível de experiência? (Por exemplo: pleno, sênior, júnior, estágio...)  junior

🔄 Processando resultados...


 📝 Essas foram as vagas encontradas! 



> Encontrei algumas vagas que correspondem ao seu perfil, confira:
> 
> 1.  **Desenvolvedor Front-End Júnior na EMPREGARE.com**
> 
>     *   **Localização:** Brasília, DF (Híbrido: 4 dias remoto e 1 dia presencial)
>     *   **Descrição:** Desenvolver interfaces de usuário eficientes e responsivas utilizando Vue.js e JavaScript, colaborar com equipes de design e back-end, manter e otimizar o código existente, participar de reuniões de equipe, realizar revisão de código e trabalhar em atividades de operação das aplicações (CI/CD) seguindo práticas de desenvolvimento Ágil com Scrum.
>     *   [**Link direto para a página da vaga**](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGbYQiLkXw02R8PIvr8NjKTLgaYVIryfHgWMrQmLaOcvSKiiQsGU9zpJrR2VH-JulGogAUIJrksATbb1yaSdG7uQIVdwbjvPCAo2nQi8lLvPXW5qCuP465jSGsORyL2WsI3EuJV3-oScdsWGLz-Bt4sVKZPymU1Q1GPZt_5rv8AXSCmMjA=)
> 
> 2.  **Desenvolvedor Front End Junior ou Pleno na Hunt RH**
> 
>     *   **Descrição:** Desenvolver e manter aplicações web utilizando Next.js e React.js, criar interfaces responsivas e dinâmicas com foco em CSS e boas práticas de UI/UX, contribuir para o desenvolvimento de aplicativos mobile com React Native, garantir performance, acessibilidade e compatibilidade entre navegadores.
>     *   **Requisitos:** Experiência sólida com Next.js e React.js, conhecimentos avançados em CSS, experiência com React Native, familiaridade com Git, compreensão de design responsivo e habilidade de trabalhar com APIs REST e/ou GraphQL.
>     *   [**Link direto para a página da vaga**](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGe-zlOw4QyO0bn5X6ZKjfvIBMtnDi6irlKTJ6SfcNSET9nbKD2fDCeXX9_kB-gUL4yB4rqU6Qwhye_pnqkSDxTvOMNmN0sFWd5H0KkTZ9RtOc3fNzcGhU2hT7UPBa_BTbjGieiatKoH1P7VDjN)
> 
> 3.  **Vaga de Desenvolvedor em Sao Leopoldo/RS - BNE**
> 
>     *   **Localização:** Sao Leopoldo/RS (Inicialmente presencial, após adaptação home office ou híbrido)
>     *   **Requisitos:** Experiência com C# .NET, noções de JavaScript, Angular ou outro UI framework, conhecimento teórico e prático de bancos de dados relacionais e linguagem SQL, capacidade de aprendizado rápido, familiaridade com endpoints REST.
>     *   [**Link direto para a página da vaga**](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHzDZkR_3n4_txbkX33_pbyvLBuRUYnKtnFHZXpC5iOEU-e-qA1zs_a2GxAmEuQFdRPFkuYwgU9jSRyU6t2Jo7gHKze7fVQ26TQSPEvZHHNuVpEh-UudnTsm77Uym9UWiy7Fagku9UvD5x34ZrK4Jzi6wL7utqJRCWUE-MfSXKxy635wus0mBBaGqIb6PaSCPAecp8Zzd-QaZanOS3rmg==)
> 
> As demais vagas encontradas nos resultados da busca não especificam o nível de experiência como júnior, mas podem ser relevantes dependendo dos seus conhecimentos e experiência.


--------------------------------------------------------------

🔄 Processando resultados de planejamento...

 🚀 Algumas dicas para decolar em seu processo seletivo! 🚀 



> Olá! Com certeza, posso te ajudar a se destacar nesses processos seletivos para desenvolvedor front-end júnior. Vamos lá:
> 
> **1. Habilidades e Experiências a Enfatizar:**
> 
> *   **Fundamentos Sólidos:** Demonstre um bom domínio de HTML, CSS e JavaScript. Mesmo sendo júnior, é importante mostrar que você entende os princípios básicos e consegue construir layouts responsivos e interativos.
> *   **Frameworks e Bibliotecas:** As vagas mencionam Vue.js, React.js e Next.js. Se você tiver experiência com algum desses, destaque isso no seu currículo e portfólio. Se não tiver, considere aprender o básico de um deles (React.js é uma boa opção por ser amplamente utilizado) e mencione isso como um objetivo de desenvolvimento.
> *   **Controle de Versão:** Familiaridade com Git é essencial. Mostre que você sabe criar branches, fazer commits e pull requests, e resolver conflitos.
> *   **UI/UX:** Mesmo sendo front-end, ter noções de UI/UX é um diferencial. Mencione se você entende sobre design responsivo, acessibilidade e usabilidade.
> *   **Soft Skills:** As empresas valorizam trabalho em equipe, comunicação e proatividade. Dê exemplos de como você demonstrou essas habilidades em projetos anteriores.
> 
> **2. Dicas de Organização para Candidaturas:**
> 
> *   **Adapte seu Currículo:** Para cada vaga, destaque as habilidades e experiências mais relevantes para aquela posição específica. Use as palavras-chave presentes na descrição da vaga.
> *   **Pesquise a Empresa:** Antes da entrevista, pesquise sobre a empresa, seus produtos, sua cultura e seus desafios. Isso mostrará seu interesse e preparo.
> 
> **3. Dicas Gerais para se Destacar no Mercado de Trabalho como Programador Front-End:**
> 
> *   **Mantenha-se Atualizado:** O mundo do front-end está sempre evoluindo. Acompanhe as novidades, faça cursos online, participe de eventos e leia blogs e artigos sobre as tecnologias que você usa e as que estão surgindo.
> *   **Construa um Portfólio:** Crie projetos pessoais para praticar suas habilidades e mostrar seu trabalho para os recrutadores. Publique seu código no GitHub e escreva artigos sobre o que você aprendeu.
> *   **Networking:** Conecte-se com outros desenvolvedores, participe de comunidades online e eventos da área. O networking pode abrir portas para oportunidades de emprego e aprendizado.
> *   **Aprimore suas Soft Skills:** Além das habilidades técnicas, as empresas valorizam cada vez mais as habilidades comportamentais, como comunicação, trabalho em equipe, liderança e resolução de problemas. Invista no desenvolvimento dessas habilidades para se destacar no mercado de trabalho.
> 
> Espero que estas dicas te ajudem a brilhar nos processos seletivos!


--------------------------------------------------------------

 Ficou com alguma dúvida? Pode contar comigo!😉 (caso não tenha dúvidas digite 'N') 

Como me destacar no mercado?

🔄 Processando resposta...


> Para se destacar no mercado de trabalho como programador front-end júnior, foque em demonstrar um bom domínio de HTML, CSS e JavaScript. Se você tiver experiência com frameworks como Vue.js, React.js ou Next.js, destaque isso em seu currículo e portfólio. Familiaridade com Git também é essencial.
> 
> Além disso, adapte seu currículo para cada vaga, destacando as habilidades e experiências mais relevantes para aquela posição e use as palavras-chave presentes na descrição da vaga. Mantenha-se atualizado com as novidades da área, construa um portfólio com projetos pessoais e faça networking com outros desenvolvedores. Aprimore também suas soft skills, como comunicação e trabalho em equipe.



 Se deseja tirar outra dúvida digite 'S'(caso não tenha, digite qualquer tecla): 

N

 Se deseja procurar outra vaga, digite 1 (para encerrar digite qualquer tecla): 

0

 Até a próxima! 😊
